In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 120)

In [2]:
ross_df = pd.read_csv('train.csv', low_memory=False)
store_df = pd.read_csv('store.csv')
test_df = pd.read_csv('test.csv')
submission_df = pd.read_csv('sample_submission.csv')

In [3]:
ross_df


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1


In [4]:
test_df

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0
...,...,...,...,...,...,...,...,...
41083,41084,1111,6,2015-08-01,1.0,0,0,0
41084,41085,1112,6,2015-08-01,1.0,0,0,0
41085,41086,1113,6,2015-08-01,1.0,0,0,0
41086,41087,1114,6,2015-08-01,1.0,0,0,0


In [5]:
store_df


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1110,1111,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1111,1112,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
1112,1113,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
1113,1114,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN


In [6]:
merged_df = ross_df.merge(store_df, how='left', on='Store') #this is gonna be the merged training dataset
merged_test_df = test_df.merge(store_df, how='left', on='Store') #this is gonna be the merged testing dataset

In [7]:
def get_date_parts(df, date_col):
    ''' return a df with date parts:
     - day of week
        - week of year
        - month
        - day of month
        - year
        - quarter
        - is_weekend
        - is_quarter_start
        - is_quarter_end
        - is_year_start
        - is_year_end
        - is_month_start
        - is_month_end
         '''
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    df['day_of_week'] = df[date_col].dt.dayofweek
    df['week_of_year'] = df[date_col].dt.weekofyear
    df['month'] = df[date_col].dt.month
    df['day_of_month'] = df[date_col].dt.day
    df['year'] = df[date_col].dt.year
    df['quarter'] = df[date_col].dt.quarter
    df['is_weekend'] = np.where(df[date_col].dt.dayofweek.isin([5,6]), 1, 0)
    df['is_quarter_start'] = df[date_col].dt.is_quarter_start.astype(int)
    df['is_quarter_end'] = df[date_col].dt.is_quarter_end.astype(int)
    df['is_year_start'] = df[date_col].dt.is_year_start.astype(int)
    df['is_year_end'] = df[date_col].dt.is_year_end.astype(int)
    df['is_month_start'] = df[date_col].dt.is_month_start.astype(int)
    df['is_month_end'] = df[date_col].dt.is_month_end.astype(int)
    return df

merged_df = get_date_parts(merged_df, 'Date')
merged_test_df = get_date_parts(merged_test_df, 'Date')

merged_df

/var/folders/yl/qjs6b9wn4zx7nh630c4my9lw0000gn/T/ipykernel_98145/3448915083.py:20: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['week_of_year'] = df[date_col].dt.weekofyear
/var/folders/yl/qjs6b9wn4zx7nh630c4my9lw0000gn/T/ipykernel_98145/3448915083.py:20: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['week_of_year'] = df[date_col].dt.weekofyear


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,day_of_week,week_of_year,month,day_of_month,year,quarter,is_weekend,is_quarter_start,is_quarter_end,is_year_start,is_year_end,is_month_start,is_month_end
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,4,31,7,31,2015,3,0,0,0,0,0,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",4,31,7,31,2015,3,0,0,0,0,0,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",4,31,7,31,2015,3,0,0,0,0,0,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN,4,31,7,31,2015,3,0,0,0,0,0,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN,4,31,7,31,2015,3,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct",1,1,1,1,2013,1,0,1,0,1,0,1,0
1017205,1112,2,2013-01-01,0,0,0,0,a,1,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN,1,1,1,1,2013,1,0,1,0,1,0,1,0
1017206,1113,2,2013-01-01,0,0,0,0,a,1,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN,1,1,1,1,2013,1,0,1,0,1,0,1,0
1017207,1114,2,2013-01-01,0,0,0,0,a,1,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN,1,1,1,1,2013,1,0,1,0,1,0,1,0


In [8]:
merged_df.groupby(by='Open')['Sales'].sum()

Open
0             0
1    5873180623
Name: Sales, dtype: int64

### Sales immer 0 wenn geschlossen --> hardcode

In [9]:
merged_df = merged_df[merged_df.Open == 1].copy() 
merged_df.groupby(by='Open')['Sales'].sum()

Open
1    5873180623
Name: Sales, dtype: int64

In [10]:
def comp_months(df):
    df = df.copy()
    df['CompetitionOpen'] = 12 * (df.year - df.CompetitionOpenSinceYear) + (df.month - df.CompetitionOpenSinceMonth)
    df['CompetitionOpen'] = df['CompetitionOpen'].map(lambda x: 0 if x < 0 else x).fillna(0)
    df = df.drop(['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear'], axis=1)
    return df

merged_df = comp_months(merged_df)
merged_test_df = comp_months(merged_test_df)

merged_df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,day_of_week,week_of_year,month,day_of_month,year,quarter,is_weekend,is_quarter_start,is_quarter_end,is_year_start,is_year_end,is_month_start,is_month_end,CompetitionOpen
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,0,NaN,NaN,NaN,4,31,7,31,2015,3,0,0,0,0,0,0,1,82.0
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",4,31,7,31,2015,3,0,0,0,0,0,0,1,92.0
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",4,31,7,31,2015,3,0,0,0,0,0,0,1,103.0
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,0,NaN,NaN,NaN,4,31,7,31,2015,3,0,0,0,0,0,0,1,70.0
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,0,NaN,NaN,NaN,4,31,7,31,2015,3,0,0,0,0,0,0,1,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016776,682,2,2013-01-01,3375,566,1,0,a,1,b,a,150.0,0,NaN,NaN,NaN,1,1,1,1,2013,1,0,1,0,1,0,1,0,76.0
1016827,733,2,2013-01-01,10765,2377,1,0,a,1,b,b,860.0,0,NaN,NaN,NaN,1,1,1,1,2013,1,0,1,0,1,0,1,0,159.0
1016863,769,2,2013-01-01,5035,1248,1,0,a,1,b,b,840.0,1,48.0,2012.0,"Jan,Apr,Jul,Oct",1,1,1,1,2013,1,0,1,0,1,0,1,0,0.0
1017042,948,2,2013-01-01,4491,1039,1,0,a,1,b,b,1430.0,0,NaN,NaN,NaN,1,1,1,1,2013,1,0,1,0,1,0,1,0,0.0


In [11]:
def check_promo_month(row):
    month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun',              
                 7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
    try:
        months = (row['PromoInterval'] or '').split(',')
        if row['Promo2Open'] and month2str[row['month']] in months:
            return 1
        else:
            return 0
    except Exception:
        return 0

def promo_cols(df):
    df = df.copy()
    # Months since Promo2 was open
    df['Promo2Open'] = 12 * (df.year - df.Promo2SinceYear) +  (df.week_of_year - df.Promo2SinceWeek)*7/30.5
    df['Promo2Open'] = df['Promo2Open'].map(lambda x: 0 if x < 0 else x).fillna(0) * df['Promo2']
    df = df.drop(['Promo2SinceYear', 'Promo2SinceWeek'], axis=1)
    # Whether a new round of promotions was started in the current month
    df['IsPromo2Month'] = df.apply(check_promo_month, axis=1) * df['Promo2']
    return df

merged_df = promo_cols(merged_df)
merged_test_df = promo_cols(merged_test_df)




In [12]:
# sales_of_last_week

In [13]:
merged_df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,PromoInterval,day_of_week,week_of_year,month,day_of_month,year,quarter,is_weekend,is_quarter_start,is_quarter_end,is_year_start,is_year_end,is_month_start,is_month_end,CompetitionOpen,Promo2Open,IsPromo2Month
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,0,NaN,4,31,7,31,2015,3,0,0,0,0,0,0,1,82.0,0.000000,0
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,1,"Jan,Apr,Jul,Oct",4,31,7,31,2015,3,0,0,0,0,0,0,1,92.0,64.131148,1
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,1,"Jan,Apr,Jul,Oct",4,31,7,31,2015,3,0,0,0,0,0,0,1,103.0,51.901639,1
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,0,NaN,4,31,7,31,2015,3,0,0,0,0,0,0,1,70.0,0.000000,0
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,0,NaN,4,31,7,31,2015,3,0,0,0,0,0,0,1,3.0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016776,682,2,2013-01-01,3375,566,1,0,a,1,b,a,150.0,0,NaN,1,1,1,1,2013,1,0,1,0,1,0,1,0,76.0,0.000000,0
1016827,733,2,2013-01-01,10765,2377,1,0,a,1,b,b,860.0,0,NaN,1,1,1,1,2013,1,0,1,0,1,0,1,0,159.0,0.000000,0
1016863,769,2,2013-01-01,5035,1248,1,0,a,1,b,b,840.0,1,"Jan,Apr,Jul,Oct",1,1,1,1,2013,1,0,1,0,1,0,1,0,0.0,1.213115,1
1017042,948,2,2013-01-01,4491,1039,1,0,a,1,b,b,1430.0,0,NaN,1,1,1,1,2013,1,0,1,0,1,0,1,0,0.0,0.000000,0


In [14]:
merged_df.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'Promo2', 'PromoInterval', 'day_of_week',
       'week_of_year', 'month', 'day_of_month', 'year', 'quarter',
       'is_weekend', 'is_quarter_start', 'is_quarter_end', 'is_year_start',
       'is_year_end', 'is_month_start', 'is_month_end', 'CompetitionOpen',
       'Promo2Open', 'IsPromo2Month'],
      dtype='object')

In [15]:
# add a new column avg_customer with the average customer of the store

def avg_customer(df):
    df=df.copy()
    avg_customer=df.groupby(by='Store')['Customers'].mean()
    avg_customer=pd.DataFrame(avg_customer)
    avg_customer=avg_customer.reset_index()
    avg_customer.columns=['Store','avg_customer']
    df=df.merge(avg_customer,how='left',on='Store')
    return df, avg_customer

def avg_customer_spent(df):
    df=df.copy()
    avg_customer_spent=df.groupby(by='Store')['Sales'].mean()/df.groupby(by='Store')['Customers'].mean()
    avg_customer_spent=pd.DataFrame(avg_customer_spent)
    avg_customer_spent=avg_customer_spent.reset_index()
    avg_customer_spent.columns=['Store','avg_customer_spent']
    df=df.merge(avg_customer_spent,how='left',on='Store')
    return df, avg_customer_spent

merged_df, avg_customer_store=avg_customer(merged_df)


In [16]:
merged_df, avg_customer_spent_store=avg_customer_spent(merged_df)

merged_df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,PromoInterval,day_of_week,week_of_year,month,day_of_month,year,quarter,is_weekend,is_quarter_start,is_quarter_end,is_year_start,is_year_end,is_month_start,is_month_end,CompetitionOpen,Promo2Open,IsPromo2Month,avg_customer,avg_customer_spent
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,0,NaN,4,31,7,31,2015,3,0,0,0,0,0,0,1,82.0,0.000000,0,564.049936,8.437366
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,1,"Jan,Apr,Jul,Oct",4,31,7,31,2015,3,0,0,0,0,0,0,1,92.0,64.131148,1,583.998724,8.482725
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,1,"Jan,Apr,Jul,Oct",4,31,7,31,2015,3,0,0,0,0,0,0,1,103.0,51.901639,1,750.077022,9.255808
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,0,NaN,4,31,7,31,2015,3,0,0,0,0,0,0,1,70.0,0.000000,0,1321.752551,7.292138
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,0,NaN,4,31,7,31,2015,3,0,0,0,0,0,0,1,3.0,0.000000,0,537.340180,8.702634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844387,682,2,2013-01-01,3375,566,1,0,a,1,b,a,150.0,0,NaN,1,1,1,1,2013,1,0,1,0,1,0,1,0,76.0,0.000000,0,1758.842887,6.372035
844388,733,2,2013-01-01,10765,2377,1,0,a,1,b,b,860.0,0,NaN,1,1,1,1,2013,1,0,1,0,1,0,1,0,159.0,0.000000,0,3403.458599,4.387680
844389,769,2,2013-01-01,5035,1248,1,0,a,1,b,b,840.0,1,"Jan,Apr,Jul,Oct",1,1,1,1,2013,1,0,1,0,1,0,1,0,0.0,1.213115,1,3081.052017,3.513703
844390,948,2,2013-01-01,4491,1039,1,0,a,1,b,b,1430.0,0,NaN,1,1,1,1,2013,1,0,1,0,1,0,1,0,0.0,0.000000,0,1712.427802,4.050834


In [17]:
merged_df=merged_df.drop(['Date','Customers','Open','PromoInterval','Promo2'],axis=1)

In [18]:
merged_df

,Store,DayOfWeek,Sales,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,day_of_week,week_of_year,month,day_of_month,year,quarter,is_weekend,is_quarter_start,is_quarter_end,is_year_start,is_year_end,is_month_start,is_month_end,CompetitionOpen,Promo2Open,IsPromo2Month,avg_customer,avg_customer_spent
0,1,5,5263,1,0,1,c,a,1270.0,4,31,7,31,2015,3,0,0,0,0,0,0,1,82.0,0.000000,0,564.049936,8.437366
1,2,5,6064,1,0,1,a,a,570.0,4,31,7,31,2015,3,0,0,0,0,0,0,1,92.0,64.131148,1,583.998724,8.482725
2,3,5,8314,1,0,1,a,a,14130.0,4,31,7,31,2015,3,0,0,0,0,0,0,1,103.0,51.901639,1,750.077022,9.255808
3,4,5,13995,1,0,1,c,c,620.0,4,31,7,31,2015,3,0,0,0,0,0,0,1,70.0,0.000000,0,1321.752551,7.292138
4,5,5,4822,1,0,1,a,a,29910.0,4,31,7,31,2015,3,0,0,0,0,0,0,1,3.0,0.000000,0,537.340180,8.702634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844387,682,2,3375,0,a,1,b,a,150.0,1,1,1,1,2013,1,0,1,0,1,0,1,0,76.0,0.000000,0,1758.842887,6.372035
844388,733,2,10765,0,a,1,b,b,860.0,1,1,1,1,2013,1,0,1,0,1,0,1,0,159.0,0.000000,0,3403.458599,4.387680
844389,769,2,5035,0,a,1,b,b,840.0,1,1,1,1,2013,1,0,1,0,1,0,1,0,0.0,1.213115,1,3081.052017,3.513703
844390,948,2,4491,0,a,1,b,b,1430.0,1,1,1,1,2013,1,0,1,0,1,0,1,0,0.0,0.000000,0,1712.427802,4.050834


In [19]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 844392 entries, 0 to 844391
Data columns (total 27 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Store                844392 non-null  int64  
 1   DayOfWeek            844392 non-null  int64  
 2   Sales                844392 non-null  int64  
 3   Promo                844392 non-null  int64  
 4   StateHoliday         844392 non-null  object 
 5   SchoolHoliday        844392 non-null  int64  
 6   StoreType            844392 non-null  object 
 7   Assortment           844392 non-null  object 
 8   CompetitionDistance  842206 non-null  float64
 9   day_of_week          844392 non-null  int64  
 10  week_of_year         844392 non-null  int64  
 11  month                844392 non-null  int64  
 12  day_of_month         844392 non-null  int64  
 13  year                 844392 non-null  int64  
 14  quarter              844392 non-null  int64  
 15  is_weekend       

In [20]:
# fill the missing values in competition distance with a high value
merged_df['CompetitionDistance']=merged_df['CompetitionDistance'].fillna(merged_df['CompetitionDistance'].max()*10)

In [21]:
merged_df

,Store,DayOfWeek,Sales,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,day_of_week,week_of_year,month,day_of_month,year,quarter,is_weekend,is_quarter_start,is_quarter_end,is_year_start,is_year_end,is_month_start,is_month_end,CompetitionOpen,Promo2Open,IsPromo2Month,avg_customer,avg_customer_spent
0,1,5,5263,1,0,1,c,a,1270.0,4,31,7,31,2015,3,0,0,0,0,0,0,1,82.0,0.000000,0,564.049936,8.437366
1,2,5,6064,1,0,1,a,a,570.0,4,31,7,31,2015,3,0,0,0,0,0,0,1,92.0,64.131148,1,583.998724,8.482725
2,3,5,8314,1,0,1,a,a,14130.0,4,31,7,31,2015,3,0,0,0,0,0,0,1,103.0,51.901639,1,750.077022,9.255808
3,4,5,13995,1,0,1,c,c,620.0,4,31,7,31,2015,3,0,0,0,0,0,0,1,70.0,0.000000,0,1321.752551,7.292138
4,5,5,4822,1,0,1,a,a,29910.0,4,31,7,31,2015,3,0,0,0,0,0,0,1,3.0,0.000000,0,537.340180,8.702634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844387,682,2,3375,0,a,1,b,a,150.0,1,1,1,1,2013,1,0,1,0,1,0,1,0,76.0,0.000000,0,1758.842887,6.372035
844388,733,2,10765,0,a,1,b,b,860.0,1,1,1,1,2013,1,0,1,0,1,0,1,0,159.0,0.000000,0,3403.458599,4.387680
844389,769,2,5035,0,a,1,b,b,840.0,1,1,1,1,2013,1,0,1,0,1,0,1,0,0.0,1.213115,1,3081.052017,3.513703
844390,948,2,4491,0,a,1,b,b,1430.0,1,1,1,1,2013,1,0,1,0,1,0,1,0,0.0,0.000000,0,1712.427802,4.050834


In [22]:
cats=["DayOfWeek", "StoreType", "Assortment", "quarter", "StateHoliday"]


In [23]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore').fit(merged_df[cats]) 
encoded_cols = list(encoder.get_feature_names_out(cats))
encoded_cols

['DayOfWeek_1',
 'DayOfWeek_2',
 'DayOfWeek_3',
 'DayOfWeek_4',
 'DayOfWeek_5',
 'DayOfWeek_6',
 'DayOfWeek_7',
 'StoreType_a',
 'StoreType_b',
 'StoreType_c',
 'StoreType_d',
 'Assortment_a',
 'Assortment_b',
 'Assortment_c',
 'quarter_1',
 'quarter_2',
 'quarter_3',
 'quarter_4',
 'StateHoliday_0',
 'StateHoliday_a',
 'StateHoliday_b',
 'StateHoliday_c']

In [24]:
merged_df[encoded_cols] = encoder.transform(merged_df[cats]) #merging the encoded columns onto out inputs dataset
merged_test_df[encoded_cols] = encoder.transform(merged_test_df[cats])

In [25]:
merged_df

,Store,DayOfWeek,Sales,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,day_of_week,week_of_year,month,day_of_month,year,quarter,is_weekend,is_quarter_start,is_quarter_end,is_year_start,is_year_end,is_month_start,is_month_end,CompetitionOpen,Promo2Open,IsPromo2Month,avg_customer,avg_customer_spent,DayOfWeek_1,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,quarter_1,quarter_2,quarter_3,quarter_4,StateHoliday_0,StateHoliday_a,StateHoliday_b,StateHoliday_c
0,1,5,5263,1,0,1,c,a,1270.0,4,31,7,31,2015,3,0,0,0,0,0,0,1,82.0,0.000000,0,564.049936,8.437366,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,2,5,6064,1,0,1,a,a,570.0,4,31,7,31,2015,3,0,0,0,0,0,0,1,92.0,64.131148,1,583.998724,8.482725,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,3,5,8314,1,0,1,a,a,14130.0,4,31,7,31,2015,3,0,0,0,0,0,0,1,103.0,51.901639,1,750.077022,9.255808,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,4,5,13995,1,0,1,c,c,620.0,4,31,7,31,2015,3,0,0,0,0,0,0,1,70.0,0.000000,0,1321.752551,7.292138,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,5,5,4822,1,0,1,a,a,29910.0,4,31,7,31,2015,3,0,0,0,0,0,0,1,3.0,0.000000,0,537.340180,8.702634,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844387,682,2,3375,0,a,1,b,a,150.0,1,1,1,1,2013,1,0,1,0,1,0,1,0,76.0,0.000000,0,1758.842887,6.372035,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
844388,733,2,10765,0,a,1,b,b,860.0,1,1,1,1,2013,1,0,1,0,1,0,1,0,159.0,0.000000,0,3403.458599,4.387680,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
844389,769,2,5035,0,a,1,b,b,840.0,1,1,1,1,2013,1,0,1,0,1,0,1,0,0.0,1.213115,1,3081.052017,3.513703,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
844390,948,2,4491,0,a,1,b,b,1430.0,1,1,1,1,2013,1,0,1,0,1,0,1,0,0.0,0.000000,0,1712.427802,4.050834,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [26]:
merged_df=merged_df.drop(cats,axis=1)


In [27]:
merged_test_df=merged_test_df.drop(cats,axis=1)

In [28]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 844392 entries, 0 to 844391
Data columns (total 44 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Store                844392 non-null  int64  
 1   Sales                844392 non-null  int64  
 2   Promo                844392 non-null  int64  
 3   SchoolHoliday        844392 non-null  int64  
 4   CompetitionDistance  844392 non-null  float64
 5   day_of_week          844392 non-null  int64  
 6   week_of_year         844392 non-null  int64  
 7   month                844392 non-null  int64  
 8   day_of_month         844392 non-null  int64  
 9   year                 844392 non-null  int64  
 10  is_weekend           844392 non-null  int64  
 11  is_quarter_start     844392 non-null  int64  
 12  is_quarter_end       844392 non-null  int64  
 13  is_year_start        844392 non-null  int64  
 14  is_year_end          844392 non-null  int64  
 15  is_month_start   

In [29]:
reduced=merged_df.sample(n=len(merged_df),random_state=1)

In [30]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

y=reduced['Sales']
X=reduced.drop(['Sales'],axis=1)

x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

xgb=XGBRegressor(n_estimators=1000,learning_rate=0.05,n_jobs=-1)
xgb.fit(x_train,y_train)





XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=-1, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [31]:
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred))))

y_pred=xgb.predict(x_test)

rmspe(y_test,y_pred)


835.5368569081573

In [32]:
merged_test_df.columns

Index(['Id', 'Store', 'Date', 'Open', 'Promo', 'SchoolHoliday',
       'CompetitionDistance', 'Promo2', 'PromoInterval', 'day_of_week',
       'week_of_year', 'month', 'day_of_month', 'year', 'is_weekend',
       'is_quarter_start', 'is_quarter_end', 'is_year_start', 'is_year_end',
       'is_month_start', 'is_month_end', 'CompetitionOpen', 'Promo2Open',
       'IsPromo2Month', 'DayOfWeek_1', 'DayOfWeek_2', 'DayOfWeek_3',
       'DayOfWeek_4', 'DayOfWeek_5', 'DayOfWeek_6', 'DayOfWeek_7',
       'StoreType_a', 'StoreType_b', 'StoreType_c', 'StoreType_d',
       'Assortment_a', 'Assortment_b', 'Assortment_c', 'quarter_1',
       'quarter_2', 'quarter_3', 'quarter_4', 'StateHoliday_0',
       'StateHoliday_a', 'StateHoliday_b', 'StateHoliday_c'],
      dtype='object')

In [33]:
merged_df.columns

Index(['Store', 'Sales', 'Promo', 'SchoolHoliday', 'CompetitionDistance',
       'day_of_week', 'week_of_year', 'month', 'day_of_month', 'year',
       'is_weekend', 'is_quarter_start', 'is_quarter_end', 'is_year_start',
       'is_year_end', 'is_month_start', 'is_month_end', 'CompetitionOpen',
       'Promo2Open', 'IsPromo2Month', 'avg_customer', 'avg_customer_spent',
       'DayOfWeek_1', 'DayOfWeek_2', 'DayOfWeek_3', 'DayOfWeek_4',
       'DayOfWeek_5', 'DayOfWeek_6', 'DayOfWeek_7', 'StoreType_a',
       'StoreType_b', 'StoreType_c', 'StoreType_d', 'Assortment_a',
       'Assortment_b', 'Assortment_c', 'quarter_1', 'quarter_2', 'quarter_3',
       'quarter_4', 'StateHoliday_0', 'StateHoliday_a', 'StateHoliday_b',
       'StateHoliday_c'],
      dtype='object')

In [34]:
test_df = merged_test_df.drop(['Date','Open','PromoInterval','Promo2'],axis=1)

In [35]:
test_df["avg_customer"] = test_df["Store"].map(avg_customer_store.set_index("Store")["avg_customer"])
test_df["avg_customer_spent"] = test_df["Store"].map(avg_customer_spent_store.set_index("Store")["avg_customer_spent"])

In [36]:
# drop Id column
test_df=test_df.drop(['Id'],axis=1)

In [37]:
# order the columns in test_df like in merged_df
test_df=test_df[X.columns]

In [63]:
test_pred=xgb.predict(test_df)

In [58]:
merged_test_df['Open'].isna().sum()

11

In [64]:
# if test_df.open == 0 or is na then sales = 0
test_pred=test_pred*(merged_test_df['Open'].fillna(1)==1)

In [65]:
test_pred.isna().sum()

0

In [66]:
submission_df=pd.DataFrame({'Id':merged_test_df['Id'],'Sales':test_pred})
submission_df

,Id,Sales
0,1,4330.295898
1,2,7553.911621
2,3,9544.389648
3,4,5999.145508
4,5,6493.348633
...,...,...
41083,41084,2990.117676
41084,41085,7977.475586
41085,41086,5700.595215
41086,41087,23165.068359


In [67]:
submission_df.to_csv('submission.csv',index=False)